In [2]:
!pip install -qU transformers einops accelerate langchain bitsandbytes pypdf sentence_transformers faiss-cpu torch xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 5.1 MB/s eta 0:00:00


In [1]:
from langchain.document_loaders import PyPDFLoader # for loading the pdf
from google.colab import drive

drive.mount('/content/gdrive')

pdf_folder_path = "/content/gdrive/My Drive/pdf-gpt/data/1512.03385.pdf"

#Use Langchain document loader to load the PDF Files
loader = PyPDFLoader(pdf_folder_path)
pages = loader.load()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

#Preprocessing using text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks = text_splitter.split_documents(pages)

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


#Get the embedding Model
document_embeddings = HuggingFaceEmbeddings()
#Create a database
db = FAISS.from_documents(text_chunks, document_embeddings)
#Create a retriever
retriever = db.as_retriever(search_kwargs={"k":3})

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline,AutoModelForCausalLM
import torch
import transformers

model_name = "tiiuae/falcon-7b-instruct" #Provide the exact model id from hugging face

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto",trust_remote_code=True, offload_folder="offload")

#Download the pre-trained model
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

# Create the pipeline
pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=100,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

#Load the pre-trained model
llm = HuggingFacePipeline(pipeline = pipeline,model_kwargs={"temperature":0.0,"device":"cuda"})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from langchain.chains import RetrievalQA

#Create RetrieverQA Object
qa_chain = RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=retriever)

In [ ]:
prompt = "Who are the authors?"
answer = qa_chain(prompt)
print(answer)